In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
fe = "035"
n_neighbors = 100

In [4]:
# =========================
# Functions
# =========================
@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    

def add_neighbor_features_first_stage(df):
    dfs = []
    columns = ['id', 'name','categories','latitude','longitude']
    #for c in columns:
    #    if c != "id":
    #        df[c] = df[c].astype(str).str.lower()

    for country, country_df in tqdm(df.groupby("country")):
        concat_df = []
        country_df = country_df.reset_index(drop=True)
        
        knn = KNeighborsRegressor(n_neighbors=min(len(country_df), n_neighbors), 
                                  metric="euclidean", n_jobs=-1)
        knn.fit(country_df[['latitude','longitude']], country_df.index)
        dists, nears = knn.kneighbors(country_df[['latitude','longitude']], return_distance=True)

        targets = country_df[TARGET].values
        for i in range(min(len(country_df), n_neighbors)):
            country_df_ = country_df[columns + ["set",TARGET]].copy()
            country_df_["rank"] = i
            country_df_[f"d_near"] = dists[:, i]
            country_df_[f"near_target"] = targets[nears[:, i]]
            for c in columns:
                country_df_[f"near_{c}"] = country_df_[c].values[nears[:, i]]
            concat_df.append(country_df_)
        concat_df = pd.concat(concat_df).reset_index(drop=True)
        dfs.append(concat_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [5]:
# =========================
# Main
# =========================
train = pd.read_csv(TRAIN_PATH)

In [6]:
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(train, train[TARGET],
                                                train[TARGET])):
    train.loc[val_idx, "set"] = i
train["set"].value_counts()

1.0    569406
0.0    569406
Name: set, dtype: int64

In [7]:
train_ = add_neighbor_features_first_stage(train.loc[train["set"] == 0].reset_index(
    drop=True))
val_ = add_neighbor_features_first_stage(train.loc[train["set"] == 1].reset_index(
    drop=True))

  0%|          | 0/209 [00:00<?, ?it/s]

  0%|          | 0/210 [00:00<?, ?it/s]

In [9]:
train = pd.concat([train_, val_]).reset_index(drop=True)

In [10]:
# 両方とも同じidは削除
train["match"] = train["id"] == train["near_id"]
train["match"] = train["match"].astype(int)
train = train[train["match"] == 0].reset_index(drop=True)

In [12]:
train["target"] = train[TARGET] == train[f"near_target"]
train["target"] = train["target"].astype(int)

In [13]:
train = reduce_mem_usage(train)
train.to_pickle(f"../output/fe/fe{fe}_{n_neighbors}near.pkl")

Memory usage of dataframe is 10516.02 MB
column =  17
0
Memory usage after optimization is: 9764.88 MB
Decreased by 7.1%
